# Basics

Let's first take a look at what's inside the ``ib_insync`` package:

In [1]:
import ib_insync
print(ib_insync.__all__)

['util', 'Object', 'ContractDetails', 'ContractDescription', 'ComboLeg', 'UnderComp', 'OrderComboLeg', 'OrderState', 'OrderStatus', 'ScannerSubscription', 'SoftDollarTier', 'Execution', 'CommissionReport', 'ExecutionFilter', 'BarData', 'RealTimeBar', 'HistogramData', 'NewsProvider', 'DepthMktDataDescription', 'AccountValue', 'RealTimeBar', 'TickData', 'TickAttribute', 'HistoricalTick', 'HistoricalTickBidAsk', 'HistoricalTickLast', 'MktDepthData', 'DOMLevel', 'BracketOrder', 'Trade', 'TradeLogEntry', 'ScanData', 'TagValue', 'PortfolioItem', 'Position', 'Fill', 'OptionComputation', 'OptionChain', 'NewsArticle', 'HistoricalNews', 'NewsTick', 'NewsBulletin', 'ConnectionStats', 'Contract', 'Stock', 'Option', 'Future', 'Forex', 'Index', 'CFD', 'Commodity', 'Bond', 'FuturesOption', 'MutualFund', 'Warrant', 'Order', 'LimitOrder', 'MarketOrder', 'StopOrder', 'StopLimitOrder', 'Ticker', 'IB', 'Client', 'Wrapper']


### Importing
The following two lines are used at the top of all notebooks. The first line imports everything and the second
starts an event loop to keep the notebook live updated:

In [2]:
from ib_insync import *
util.startLoop()

### Connecting
The main player of the whole package is the "IB" class. Let's create an IB instance and connect to a running TWS/IBG application (remember it must have an open API port):

In [3]:
ib = IB()
ib.connect('127.0.0.1', 7497, clientId=10)

If the connection failed, then double-check the hostname and port. For IB Gateway the default port is 4001. Make sure the clientId is not already in use.

If the connection succeeded, then ``ib`` will be synchronized with TWS/IBG. The "current state" is now available via methods such as ib.positions(), ib.trades(), ib.openTrades(), ib.accountValues() or ib.tickers(). For example, to see the actual net liquidation value:

In [4]:
[v for v in ib.accountValues() if v.tag == 'NetLiquidationByCurrency' and v.currency == 'BASE']

[AccountValue(account='DU15100', tag='NetLiquidationByCurrency', value='500000', currency='BASE')]

This state will automatically be kept in sync with TWS/IBG. So an order fill will be added as soon as it is reported, or account values will be updated as soon as they change in TWS.

### Contracts

Contracts can be specified in different ways:
* The ibapi way, by creating an empty Contract object and setting it's attributes one by one;
* By using Contract and giving the attributes as keyword argument;
* By using the specialized Stock, Option, Future, Forex, Index, CFD, Commodity,
  Bond, FuturesOption, MutualFund or Warrant contracts.

Some examples:

In [5]:
Contract(conId=270639)
Stock('AMD', 'SMART', 'USD')
Stock('INTC', 'SMART', 'USD', primaryExchange='NASDAQ')
Forex('EURUSD')
CFD('IBUS30')
Future('ES', '20180921', 'GLOBEX')
Option('SPY', '20170721', 240, 'C', 'SMART')
Bond(secIdType='ISIN', secId='US03076KAA60');

### Sending a request

The IB class has nearly all request methods that the IB API offers. The methods that return a result will block until finished and then return the result. Take for example reqContractDetails:

In [6]:
%%time

contract = Stock('TSLA', 'SMART', 'USD')
cds = ib.reqContractDetails(contract)

CPU times: user 1.07 ms, sys: 5 µs, total: 1.08 ms
Wall time: 224 ms


Requesting contract details generally takes about 100 - 300 ms. Let's have a look at the actual
contract details that were sent:

In [7]:
cds

[ContractDetails(summary=Contract(conId=76792991, symbol='TSLA', secType='STK', exchange='SMART', primaryExchange='NASDAQ', currency='USD', localSymbol='TSLA', tradingClass='NMS'), marketName='NMS', minTick=0.01, orderTypes='ACTIVETIM,ADJUST,ALERT,ALGO,ALLOC,AON,AVGCOST,BASKET,COND,CONDORDER,DARKPOLL,DAY,DEACT,DEACTDIS,DEACTEOD,DIS,GAT,GTC,GTD,GTT,HID,IBDARK,ICE,IMB,IOC,LIT,LMT,LOC,MIT,MKT,MOC,MTL,NONALGO,OCA,OPG,OPGREROUT,PEGBENCH,POSTONLY,PREOPGRTH,REL,RPI,RTH,SCALE,SCALERST,STP,STPLMT,SWEEP,TIMEPRIO,TRAIL,TRAILLIT,TRAILLMT,TRAILMIT,WHATIF', validExchanges='SMART,AMEX,CHX,ARCA,ISLAND,DRCTEDGE,NSX,BEX,BATS,EDGEA,BYX,PSX', priceMagnifier=1, longName='TESLA INC', industry='Consumer, Cyclical', category='Auto Manufacturers', subcategory='Auto-Cars/Light Trucks', timeZoneId='EST5EDT', tradingHours='20170807:0005-2358;20170808:0005-2358;20170809:0005-2358;20170810:0005-2358;20170811:0005-2358;20170812:0005-2358;20170813:0005-2358;20170814:0005-2358;20170815:0005-2358;20170816:0005-2358;201

### Logging

The following will put log messages of INFO and higher level under the current active cell:

In [8]:
util.logToConsole()

To see all debug messages (including network traffic):

In [9]:
import logging
util.logToConsole(logging.DEBUG)

### Disconnecting

The following will disconnect ``ib`` and clear all its state:

In [10]:
ib.disconnect()

INFO:ib_insync.ib:Disconnecting from 127.0.0.1:7497, 128 B sent in 7 messages, 8.10 kB received in 153 messages, session time 789 ms.
2017-08-07 14:06:30,918 ib_insync.ib INFO Disconnecting from 127.0.0.1:7497, 128 B sent in 7 messages, 8.10 kB received in 153 messages, session time 789 ms.
2017-08-07 14:06:30,918 ib_insync.ib INFO Disconnecting from 127.0.0.1:7497, 128 B sent in 7 messages, 8.10 kB received in 153 messages, session time 789 ms.
INFO:ib_insync.client:Disconnected
2017-08-07 14:06:30,923 ib_insync.client INFO Disconnected
2017-08-07 14:06:30,923 ib_insync.client INFO Disconnected
